<a href="https://colab.research.google.com/github/Fazira2025/weatherwise-template/blob/main/starter_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – Starter Notebook

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---


In [ ]:
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install ipywidgets


## 📦 Setup and Configuration
Import required packages and setup environment.

In [2]:
import requests
import matplotlib.pyplot as plt
import re
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, clear_output

## 🌤️ Weather Data Functions

In [3]:
def get_weather_data(location, format_type='plaintext'):
    """
    Get weather data from wttr.in API
    Args:
        location (str): City name or location
    Returns:
        str: Weather data in text format or None if an error occurs
    Raises:
        Various requests exceptions which are caught and printed
    """
    try:
        if format_type.lower() == "json":
            url = f'https://wttr.in/{location}?format=j1'
        else:
            url = f'https://wttr.in/{location}'

        response = requests.get(url, timeout=10)
        response.raise_for_status()

        if format_type.lower() == "json":
            return response.json()
        else:
            print(response.text)
            return None
    except requests.exceptions.HTTPError as errh:
        print(f"HTTP Error: {errh}")
    except requests.exceptions.ConnectionError as errc:
        print(f"Connection Error: {errc}")
    except requests.exceptions.Timeout as errt:
        print(f"Timeout Error: {errt}")
    except requests.exceptions.TooManyRedirects as errr:
        print(f"Redirect Error: {errr}")
    except requests.exceptions.RequestException as err:
        print(f"Unknown Error: {err}")

    return None

## 📊 Visualisation Functions

In [4]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def get_emoji(weather_desc):
    """
    Return an emoji corresponding to the weather description.
    """
    weather_lower = weather_desc.lower()

    rain_keywords = ['rain', 'drizzle', 'shower', 'thundery outbreaks', 'downpour', 'wet']
    snow_keywords = ['snow', 'blizzard', 'sleet', 'ice', 'hail', 'flurry']
    cloud_keywords = ['cloud', 'overcast', 'mist', 'fog', 'haze', 'partly cloudy', 'scattered clouds']
    fog_keywords = ['mist', 'fog', 'haze']
    sun_keywords = ['sunny', 'clear', 'fair', 'hot', 'bright', 'clear skies', 'clear weather']
    wind_keywords = ['wind', 'gale', 'storm', 'breeze', 'gust']

 # Rain conditions
    if any(word in weather_lower for word in rain_keywords):
        if 'thunder' in weather_lower or 'storm' in weather_lower or 'thundery outbreaks' in weather_lower:
            return '⛈️'  # Thunderstorm
        elif 'light' in weather_lower or 'patchy' in weather_lower:
            return '🌦️'  # Sun behind rain cloud
        else:
            return '🌧️'  # Rain cloud

    # Snow conditions
    elif any(word in weather_lower for word in snow_keywords):
        if 'light' in weather_lower or 'patchy' in weather_lower:
            return '🌨️'  # Snow cloud
        else:
            return '❄️'  # Snowflake

    # Cloud conditions
    elif any(word in weather_lower for word in cloud_keywords):
        if 'fog' in weather_lower or 'mist' in weather_lower:
            return '🌫️'  # Fog
        elif 'overcast' in weather_lower:
            return '☁️'  # Cloudy weather
        elif 'partly' in weather_lower or 'scattered' in weather_lower:
            return '⛅'  # Sun behind cloud
        else:
            return '☁️'  # Cloud

    # Clear conditions (sunny/fair weather)
    elif any(word in weather_lower for word in sun_keywords):
        return '☀️'  # Sun

    # Wind conditions
    elif any(word in weather_lower for word in wind_keywords):
        return '💨'  # Wind blowing

    # Fog conditions
    elif any(word in weather_lower for word in fog_keywords):
        return '🌫️'  # Wind blowing

    # Default (unknown or undefined weather)
    else:
        return '🌡️'  # Thermometer (default)

def create_temperature_visualisation(location, target_date):
    data = get_weather_data(location, format_type="json")
    if data is None:
        print("Failed to retrieve weather data.")
        return None

    try:
        weather_days = data['weather']  # List of daily forecasts
    except KeyError:
        print("Unexpected data format.")
        return None

    target_date_str = target_date.strftime("%Y-%m-%d")

    for day in weather_days:
        if day.get('date') == target_date_str:
            date = day.get('date', 'Unknown Date')
            try:
                dt_obj = datetime.strptime(date, "%Y-%m-%d")
                formatted_date = dt_obj.strftime("%A, %d %B %Y")
            except Exception:
                formatted_date = date

            weather_desc = day.get('hourly', [{}])[4].get('weatherDesc', [{'value': 'N/A'}])[0]['value']
            mintempC = day.get('mintempC', 'N/A')
            maxtempC = day.get('maxtempC', 'N/A')
            avgtempC = day.get('avgtempC', 'N/A')
            weather_emoji = get_emoji(weather_desc)

            # Return data dictionary instead of printing
            return {
                "location": location.capitalize(),
                "date": formatted_date,
                "weather_desc": weather_desc,
                "weather_emoji": weather_emoji,
                "min_temp": mintempC,
                "max_temp": maxtempC,
                "avg_temp": avgtempC
            }
    print(f"No weather data found for {target_date_str}.")
    return None

In [5]:
def create_precipitation_visualisation(location, day_index=0):
    data = get_weather_data(location, 'json')
    if not data:
        print("Failed to get data.")
        return

    hourly = data['weather'][day_index]['hourly']
    times = [f"{int(h['time'])//100:02d}:00" for h in hourly]
    precip = [float(h['precipMM']) for h in hourly]
    rain_chance = [int(h['chanceofrain']) for h in hourly]

    fig, ax1 = plt.subplots()

    ax1.plot(times, precip, 'b-o', label='Precipitation (mm)')
    ax1.set_xlabel('24 Hour')
    ax1.set_ylabel('Precipitation (mm)', color='b')
    ax1.tick_params(axis='y', labelcolor='b')

    ax2 = ax1.twinx()
    ax2.plot(times, rain_chance, 'g--x', label='Chance of Rain (%)')
    ax2.set_ylabel('Chance of Rain (%)', color='g')
    ax2.tick_params(axis='y', labelcolor='g')

    if day_index == 0:
        days = 'Today'
    elif day_index == 1:
        days = 'Tomorrow'
    else:
        days = 'After-Tomorrow'

    plt.title(f'{days} Graphical Representation of Precipitation and Rainfall Probability at {location.capitalize()}')
    plt.grid(True)
    plt.show()

## 🤖 Natural Language Processing

In [6]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question, location):
    # Define regex pattern to detect time
    time_pattern = r"(today|tomorrow|after[-\s]?tomorrow|day[-\s]?after[-\s]?tomorrow)"

    # Lists of outdoor and indoor activities
    outdoor_activities = r"(run|swim|sports|tennis|cycling|hiking|jogging|fishing|camping)"
    indoor_activities = r"(cooking|reading|studying|watching movies|baking|painting|knitting|gaming)"

    # Search for time in the question
    time_match = re.search(time_pattern, question, re.IGNORECASE)
    # Search for outdoor activity
    outdoor_match = re.search(outdoor_activities, question, re.IGNORECASE)
    # Search for indoor activity
    indoor_match = re.search(indoor_activities, question, re.IGNORECASE)

    # Default time to "today" if not found
    time = "today"
    activity = None  # Default no activity found
    activity_type = None  # Default no activity type found

    if time_match:
        time = time_match.group(1).lower()  # Extract time (today, tomorrow, after-tomorrow)

    if outdoor_match:
        activity = outdoor_match.group(1).lower()
        activity_type = "Outdoor"
    elif indoor_match:
        activity = indoor_match.group(1).lower()
        activity_type = "Indoor"

    # Calculate target date based on time
    target_date = datetime.now()
    if time == "tomorrow":
        target_date += timedelta(days=1)
    elif time in ["after-tomorrow", "day after tomorrow"]:
        target_date += timedelta(days=2)

    # Call function to display weather based on location and date
    if location and activity and activity_type:
        weather = create_temperature_visualisation(location, target_date)
        if weather is None:
            print("Weather data not available.")
            return

        response = generate_weather_response(
            location=weather['location'],
            date_str=weather['date'],
            activity=activity,
            activity_type=activity_type,
            weather_desc=weather['weather_desc'],
            weather_emoji=weather['weather_emoji'],
            min_temp=weather['min_temp'],
            max_temp=weather['max_temp'],
            avg_temp=weather['avg_temp'],
            time = time
        )
        print(response)
    else:
        print("Could not detect location, activity, or time properly.")

## 🧭 User Interface

In [7]:
header_html = widgets.HTML(
    value="<pre style='color:blue; font-weight:bold;'>"
          "============================================\n"
          "               Weather Wise                 \n"
          "============================================"
          "</pre>"
)

# Inputs for user info
name_input = widgets.Text(description='Name:', placeholder='Enter your name')
location_input = widgets.Text(description='Location:', placeholder='Enter your location')
check_button = widgets.Button(description='Check', layout=widgets.Layout(margin='10px 0 0 90px'))

output = widgets.Output()

# Container for name/location inputs and check button
input_box = widgets.VBox([name_input, location_input, check_button])

# Container for main menu input and button (starts hidden)
menu_choice_input = widgets.Text(description='Choice:', placeholder='Enter 1-5')
menu_submit_button = widgets.Button(description='Submit')
menu_input_box = widgets.HBox([menu_choice_input, menu_submit_button])
menu_input_box.layout.display = 'none'  # hide initially

# Back button for submenus (hidden initially)
back_button = widgets.Button(description='Back to Main Menu', layout=widgets.Layout(margin='10px 0 0 90px'))
back_button.layout.display = 'none'

def show_menu(name, location):
    with output:
        clear_output()
        print(f"Welcome, {name} what will you check about {location}!\n")
        print("Main Menu:")
        print(f"1. Check Current Weather & View Weather Forecast at {location}")
        print(f"2. Check Current Weather Hourly at {location}")
        print("3. Activities Suggestions & Weather Prediction assistant ")
        print("4. Check Other Location")
        print("5. Exit Program")
    menu_input_box.layout.display = ''
    back_button.layout.display = 'none'

def show_option_1(name, location):
    with output:
        clear_output()
        print(f"Wait a second for a display:: Current Weather & Forecast for {location}\n")
        get_weather_data(location)
    menu_input_box.layout.display = 'none'
    back_button.layout.display = ''

option_2_box = widgets.VBox()
option_2_box.layout.display = 'none'

def show_option_2(name, location):
    with output:
        clear_output(wait=True)
        print(f"Wait a second for a display: Current Weather Hourly for {location}\n")

    menu_input_box.layout.display = 'none'
    back_button.layout.display = ''

    day_dropdown = widgets.Dropdown(
        options=[('Today', 'today'), ('Tomorrow', 'tomorrow'), ('After Tomorrow', 'after_tomorrow')],
        description='Pilih hari:',
        value='today'
    )

    def on_day_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            selected = change['new']
            with output:
                clear_output(wait=True)
                print(f"Wait a second for a display:: {selected.capitalize()} Weather Hourly for {location}\n")

                if selected == 'today':
                    day_index = 0
                elif selected == 'tomorrow':
                    day_index = 1
                else:
                    day_index = 2
                create_precipitation_visualisation(location, day_index=day_index)

    day_dropdown.observe(on_day_change)


    option_2_box.children = [day_dropdown]
    option_2_box.layout.display = ''

def show_option_3(name, location):
    with output:
        clear_output()
        print(f"Wait a second for a display:: Activities Suggestions & Weather Prediction for {location}\n")
        question_text = widgets.Text(description="Question:", placeholder='ex: How about running today?')
        submit_q = widgets.Button(description="Submit", layout=widgets.Layout(margin='10px 0 0 90px'))

        def on_submit_q(btn):
            with output:
                clear_output()
                question = question_text.value
                parse_weather_question(question, location)

        submit_q.on_click(on_submit_q)
        display(widgets.Label("What Activities & When (Today, Tomorrow, Day After Tomorrow)?"))
        display(question_text, submit_q)

    menu_input_box.layout.display = 'none'
    back_button.layout.display = ''

def show_option_4(name, location):
    with output:
        clear_output()
        print(f"Option 4: Update Your Location (Current: {location})\n")

        new_location = widgets.Text(description='🤖', placeholder='Enter New Location')
        update_btn = widgets.Button(description="Update Location", layout=widgets.Layout(margin='10px 0 0 90px'))

        def on_update_location(btn):
            loc = new_location.value.strip()
            with output:
                clear_output()
                if not loc:
                    print("⚠️ Please enter a location before updating.")
                    display(widgets.VBox([new_location, update_btn]))
                    return
                else:
                    user_info['location'] = loc
                    print(f"📍 Your New Location: {user_info['location']}")
                    show_menu(name, user_info['location'])

        update_btn.on_click(on_update_location)

        display(widgets.VBox([new_location, update_btn]))

    menu_input_box.layout.display = 'none'
    back_button.layout.display = ''

def on_check_clicked(b):
    name = name_input.value.strip()
    location = location_input.value.strip()
    if not name or not location:
        with output:
            clear_output()
            print("Please enter both your name and location.")
        return

    # Hide user info inputs and button
    input_box.layout.display = 'none'
    show_menu(name, location)

def on_menu_submit_clicked(b):
    choice = menu_choice_input.value.strip()
    name = name_input.value.strip()
    location = location_input.value.strip()
    if choice not in ['1','2','3','4','5']:
        with output:
            clear_output()
            print("Invalid choice. Please enter a number between 1 and 5.\n")
            show_menu(name, location)
        return
    if choice == '5':
        with output:
            clear_output()
            print(f"Thank you for using Weather Wise. Goodbye! {name}")
        menu_input_box.layout.display = 'none'
        back_button.layout.display = 'none'
        return
    if choice == '1':
        show_option_1(name, location)
    elif choice == '2':
        show_option_2(name, location)
    elif choice == '3':
        show_option_3(name, location)
    elif choice == '4':
        show_option_4(name, location)
    else:
        with output:
            clear_output()
            print(f"Option {choice} belum diimplementasikan.")
            show_menu(name, location)
    menu_choice_input.value = ''

def on_back_clicked(b):
    global option_2_box
    option_2_box.layout.display = 'none'

    name = name_input.value.strip()
    location = location_input.value.strip()
    show_menu(name, location)

check_button.on_click(on_check_clicked)
menu_submit_button.on_click(on_menu_submit_clicked)
back_button.on_click(on_back_clicked)

## 🧩 Main Application Logic

In [8]:
# Tie everything together here
def generate_weather_response(location,time, date_str, activity, activity_type, weather_desc, weather_emoji, min_temp, max_temp, avg_temp):
    desc = weather_desc.lower()
    avg_temp = float(avg_temp)

    bad_weather_keywords = ['rain', 'thunder', 'fog', 'mist', 'shower', 'snow', 'drizzle', 'downpour', 'blizzard', 'sleet', 'ice', 'hail', 'flurry']

    bad_weather = any(word in desc for word in bad_weather_keywords)

    if activity_type.lower() == "outdoor":
        if bad_weather or avg_temp < 15:
            suggestion = (f"The weather is bad or cold ({avg_temp}°C), so outdoor activities like {activity} are not recommended. Better to stay indoors.")
        elif 15 <= avg_temp <= 30:
            suggestion = (f"The weather and temperature ({avg_temp}°C) are comfortable, perfect for outdoor activities like {activity}.")
        else:
            suggestion = (f"It's very hot ({avg_temp}°C). Even if weather is good, be careful doing outdoor activities like {activity}. Indoor activities are recommended.")
    else:  # indoor
        if bad_weather or avg_temp < 15:
            suggestion = (f"The weather is bad or cold ({avg_temp}°C), so indoor activities like {activity} are the best choice.")
        else:
            suggestion = (f"The weather is good ({avg_temp}°C). Indoor activities like {activity} are fine, but you could also try outdoor activities like jogging or basketball.")

    return (
        "\n" + "-" * 53
        +"\n" + " " * 20 + "ACTIVITY SUGGESTION"
        +"\n" + "-" * 53
        +"\n" + f"📍 Location   : {location}"
        +"\n" + f"📅 Date       : {time.capitalize()}, {date_str}"
        +"\n" + f"🏃 Activity   : ({activity_type.capitalize()}) - {activity.capitalize()}"
        +"\n" + f"🌤️ Weather    : {weather_emoji} - {weather_desc}"
        +"\n" + f"🌡️ Temperature: Min {min_temp}°C | Max {max_temp}°C | Avg {avg_temp}°C"
        +"\n"
        +"\n" + "✅ Suggestion:"
        +"\n" + f"{time.capitalize()}, " +suggestion
        +"\n" +"-" * 53 + "\n"
    )



## 🧪 Testing and Examples

In [9]:
  display(header_html, input_box, output, menu_input_box, back_button, option_2_box)

HTML(value="<pre style='color:blue; font-weight:bold;'>============================================\n         …

Output()

Button(description='Back to Main Menu', layout=Layout(display='none', margin='10px 0 0 90px'), style=ButtonSty…

VBox(layout=Layout(display='none'))

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.